# Time Series Prediction 

This notebook will be dedicated to stock price prediction 

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_squared_error as mse
import os

In [7]:

folder_path = "stock-price-predictions/price/"

dataframes = {}

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        dataframes[filename[:-4]] = df

for name, df in dataframes.items():
    print(f"Loaded {name}: {df.shape}")


Loaded CSCO: (1258, 7)
Loaded BUD: (1258, 7)
Loaded SPLP: (1258, 7)
Loaded BA: (1258, 7)
Loaded MDT: (1258, 7)
Loaded HSBC: (1258, 7)
Loaded TM: (1258, 7)
Loaded V: (1258, 7)
Loaded MO: (1258, 7)
Loaded NVS: (1258, 7)
Loaded BBL: (1258, 7)
Loaded BCH: (1258, 7)
Loaded CHTR: (1258, 7)
Loaded C: (1258, 7)
Loaded T: (1258, 7)
Loaded CHL: (1258, 7)
Loaded SNP: (1258, 7)
Loaded UTX: (1258, 7)
Loaded BAC: (1258, 7)
Loaded BP: (1258, 7)
Loaded UN: (1258, 7)
Loaded PEP: (1258, 7)
Loaded IEP: (1258, 7)
Loaded UL: (1258, 7)
Loaded D: (1258, 7)
Loaded MRK: (1258, 7)
Loaded TSM: (1258, 7)
Loaded CODI: (1258, 7)
Loaded ABBV: (1177, 7)
Loaded GMRE: (297, 7)
Loaded ORCL: (1258, 7)
Loaded PG: (1258, 7)
Loaded CAT: (1258, 7)
Loaded MCD: (1258, 7)
Loaded AMZN: (1258, 7)
Loaded INTC: (1258, 7)
Loaded MMM: (1258, 7)
Loaded KO: (1258, 7)
Loaded NEE: (1258, 7)
Loaded UPS: (1258, 7)
Loaded MSFT: (1258, 7)
Loaded TOT: (1258, 7)
Loaded EXC: (1258, 7)
Loaded HD: (1258, 7)
Loaded SO: (1258, 7)
Loaded XOM: (1258,

In [8]:
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2012-09-04,27.327276,27.606695,27.243450,27.578754,21.952684,13438500
1,2012-09-05,27.653267,27.755720,27.476299,27.606695,21.974926,6460300
2,2012-09-06,27.392473,27.392473,27.150311,27.206194,21.922386,5406200
3,2012-09-07,27.252764,27.336590,27.140995,27.234137,21.944904,2729500
4,2012-09-10,27.327276,27.327276,27.131681,27.131681,21.862347,2729300
...,...,...,...,...,...,...,...
1253,2017-08-28,39.610001,39.680000,39.299999,39.529999,39.529999,2119900
1254,2017-08-29,39.590000,39.650002,39.360001,39.380001,39.380001,1443900
1255,2017-08-30,39.270000,39.419998,39.110001,39.180000,39.180000,1825100
1256,2017-08-31,39.259998,39.320000,39.169998,39.240002,39.240002,2086200


In [10]:

folder_path = 'stock-price-predictions/price/'

volatility_dict = {}

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        
        if {'High', 'Low', 'Close'}.issubset(df.columns):
            df['Volatility'] = df['Close'].std()
            #df['Volatility'] = (df['High'] - df['Low']) / df['Close']
            avg_volatility = df['Volatility'].mean()
            volatility_dict[filename[:-4]] = avg_volatility

sorted_volatility = sorted(volatility_dict.items(), key=lambda x: x[1], reverse=True)

for stock, vol in sorted_volatility:
    print(f"{stock}: {vol:.5f}")


BRK-A: 33823.27304
PCLN: 327.81740
AMZN: 234.49625
GOOG: 164.42096
CHTR: 78.33777
LMT: 59.73121
FB: 40.25637
UNH: 39.06597
GD: 38.67045
BA: 32.94518
MMM: 30.64895
HD: 28.84244
AMGN: 28.15103
CELG: 26.96468
PTR: 26.80493
AAPL: 25.76712
REX: 24.39223
IEP: 22.96054
BABA: 22.57322
MA: 21.00870
NEE: 20.34242
DIS: 20.00740
HON: 18.97971
V: 17.69743
MCD: 17.38828
BHP: 17.21948
JNJ: 16.08153
BBL: 15.64786
MDT: 13.76360
MO: 13.69414
TM: 13.36761
SRE: 13.07403
DHR: 13.01577
JPM: 12.94966
PEP: 12.76380
CVX: 12.45644
MSFT: 12.43517
BUD: 12.42005
CAT: 11.85779
UTX: 11.68321
NVS: 11.23771
SLB: 11.16909
RDS-B: 11.07431
UPS: 10.79303
PM: 10.19438
SNP: 9.77833
BCH: 9.61492
ABBV: 9.12106
PCG: 8.74342
AEP: 8.54496
D: 7.69747
HSBC: 7.56990
C: 7.20688
WFC: 6.91445
XOM: 6.81195
TOT: 6.78000
NGG: 6.67582
MRK: 6.57171
DUK: 6.35012
CMCSA: 5.96339
PG: 5.92297
WMT: 5.91624
BP: 5.89641
TSM: 5.66369
CHL: 5.40105
INTC: 5.37458
PICO: 5.10775
SNY: 4.92004
UN: 4.85781
ORCL: 4.52742
UL: 4.38881
CSCO: 4.06528
BAC: 3.776

In [12]:

# Analyzing APPL


